In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn
from acquire import get_titanic_data
from prepare import chop_data
from prepare import clean_titanic

this file exists, reading csv
this file exists, reading csv
this file exists, reading csv


In [2]:
from acquire import get_titanic_data

In [52]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [68]:
from sklearn.linear_model import LogisticRegression

In [4]:

def preprocess_titanic(train, validate, test):
    appendo = []
  
    for stuff in [train, test, validate]:
        stuff['gender'] = stuff['sex'].map({'male': 0, 'female': 1})
        stuff.drop(columns='sex', inplace=True)
        stuff.drop(columns='passenger_id', inplace=True)
        stuff["pclass"] = stuff["pclass"].astype(int)
        stuff['embark_town'].fillna('missing', inplace=True)
        new_embark_town = pd.get_dummies(stuff['embark_town'], drop_first=True).astype(int)
        appendo.append(pd.concat([stuff,new_embark_town],axis=1).drop(columns='embark_town'))
        

    return appendo

In [5]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier,export_text, plot_tree

In [6]:
boaty = clean_titanic(get_titanic_data)

In [7]:
train_boaty,validate_boaty,test_boaty= chop_data(boaty,"survived")

In [8]:
train_boaty

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone
776,776,0,3,male,0,0,7.7500,Queenstown,1
829,829,1,1,female,0,0,80.0000,Southampton,1
215,215,1,1,female,1,0,113.2750,Cherbourg,0
258,258,1,1,female,0,0,512.3292,Cherbourg,1
129,129,0,3,male,0,0,6.9750,Southampton,1
...,...,...,...,...,...,...,...,...,...
125,125,1,3,male,1,0,11.2417,Cherbourg,0
360,360,0,3,male,1,4,27.9000,Southampton,0
55,55,1,1,male,0,0,35.5000,Southampton,1
298,298,1,1,male,0,0,30.5000,Southampton,1


In [9]:
trained,validated,tested = preprocess_titanic(train_boaty,validate_boaty,test_boaty)

In [10]:
validated

,survived,pclass,sibsp,parch,fare,alone,gender,Queenstown,Southampton
763,1,1,1,2,120.0000,0,1,0,1
112,0,3,0,0,8.0500,1,0,0,1
230,1,1,1,0,83.4750,0,1,0,1
500,0,3,0,0,8.6625,1,0,0,1
237,1,2,0,2,26.2500,0,1,0,1
...,...,...,...,...,...,...,...,...,...
883,0,2,0,0,10.5000,1,0,0,1
149,0,2,0,0,13.0000,1,0,0,1
843,0,3,0,0,6.4375,1,0,0,0
581,1,1,1,1,110.8833,0,1,0,0


In [ ]:
trained.survived.mode() # This is telling you the baseline that appeared most, didnt survive

In [11]:
(trained.survived == 0).mean() #ACCURACY BASELINE

0.6161048689138576

In [ ]:
#Baseline is 0 and accuracy is 61.6%

In [12]:
X_train = trained.drop(columns="survived") #Create X_train variables that drop the survive target
X_validate = validated.drop(columns='survived')
X_test = tested.drop(columns='survived')

In [13]:
y_train = trained.survived #Create y target variables
y_validate = validated.survived
y_test = tested.survived

In [ ]:
#Create decision tree object

In [14]:
classy = DecisionTreeClassifier()

In [15]:
classy.fit(X_train,y_train) #Fit the model using TRAIN DATA ONLY!

DecisionTreeClassifier()

In [17]:
#Create your predictor!
y_predictor = classy.predict(X_train)

In [18]:
#TEST PREDICTOR
y_predictor[:10]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

In [16]:
classy.score(X_train,y_train) #MODEL SCORE

0.947565543071161

In [ ]:
#you can normalize with index for pd.crosstab for confusion matrix

In [25]:
pd.crosstab(y_train, y_predictor,normalize="index")

col_0,0,1
survived,,
0,0.987842,0.012158
1,0.117073,0.882927


In [27]:
confusion_matrix(y_train, y_predictor) #CONFUSION MATRIX rows, columns

array([[325,   4],
       [ 24, 181]])

In [28]:
print(classification_report(y_train,y_predictor))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       329
           1       0.98      0.88      0.93       205

    accuracy                           0.95       534
   macro avg       0.95      0.94      0.94       534
weighted avg       0.95      0.95      0.95       534



In [ ]:
trained.survived.mode

In [ ]:
#Compute: Accuracy, true positive rate, false positive rate, 
#true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
#Determine positive =1 (survived)

In [34]:
pd.crosstab(y_train,y_predictor)

col_0,0,1
survived,,
0,325,4
1,24,181


In [35]:
breakdown = pd.crosstab(y_train,y_predictor) #ASSIGN CROSSTAB

In [36]:
tp = breakdown.iloc[1,1]
tn = breakdown.iloc[0,0]
fp = breakdown.iloc[0,1]
fn = breakdown.iloc[1,0]

In [37]:
tp, tn, fp, fn #CHECK COUNTS OF DATA

(181, 325, 4, 24)

In [40]:
def compute_class_metrics(y_train, y_pred):
    
    counts = pd.crosstab(y_train, y_pred)
    TP = counts.iloc[1,1]
    TN = counts.iloc[0,0]
    FP = counts.iloc[0,1]
    FN = counts.iloc[1,0]
    
    
    all_ = (TP + TN + FP + FN)

    accuracy = (TP + TN) / all_

    TPR = recall = TP / (TP + FN)
    FPR = FP / (FP + TN)

    TNR = TN / (FP + TN)
    FNR = FN / (FN + TP)

    precision =  TP / (TP + FP)
    f1 =  2 * ((precision * recall) / ( precision + recall))

    support_pos = TP + FN
    support_neg = FP + TN
    
    print(f"Accuracy: {accuracy}\n")
    print(f"True Positive Rate/Sensitivity/Recall/Power: {TPR}")
    print(f"False Positive Rate/False Alarm Ratio/Fall-out: {FPR}")
    print(f"True Negative Rate/Specificity/Selectivity: {TNR}")
    print(f"False Negative Rate/Miss Rate: {FNR}\n")
    print(f"Precision/PPV: {precision}")
    print(f"F1 Score: {f1}\n")
    print(f"Support (0): {support_pos}")
    print(f"Support (1): {support_neg}")

In [42]:
compute_class_metrics(y_train, y_predictor)

Accuracy: 0.947565543071161

True Positive Rate/Sensitivity/Recall/Power: 0.8829268292682927
False Positive Rate/False Alarm Ratio/Fall-out: 0.0121580547112462
True Negative Rate/Specificity/Selectivity: 0.9878419452887538
False Negative Rate/Miss Rate: 0.11707317073170732

Precision/PPV: 0.9783783783783784
F1 Score: 0.9282051282051282

Support (0): 205
Support (1): 329


In [43]:
pd.crosstab(y_train, y_predictor, normalize='index')

col_0,0,1
survived,,
0,0.987842,0.012158
1,0.117073,0.882927


In [44]:
for x in range(1,21):
    #create the object
    tree = DecisionTreeClassifier(max_depth=x)
    
    #fit the object
    tree.fit(X_train, y_train) #fit only on TRAIN DATA!
    
    #calculate the accuracy
    acc = tree.score(X_train, y_train)
    
    print(f'with a max depth of {x}, the accuracy is {round(acc,2)}')

with a max depth of 1, the accuracy is 0.8
with a max depth of 2, the accuracy is 0.8
with a max depth of 3, the accuracy is 0.81
with a max depth of 4, the accuracy is 0.82
with a max depth of 5, the accuracy is 0.86
with a max depth of 6, the accuracy is 0.88
with a max depth of 7, the accuracy is 0.9
with a max depth of 8, the accuracy is 0.92
with a max depth of 9, the accuracy is 0.93
with a max depth of 10, the accuracy is 0.94
with a max depth of 11, the accuracy is 0.94
with a max depth of 12, the accuracy is 0.95
with a max depth of 13, the accuracy is 0.95
with a max depth of 14, the accuracy is 0.95
with a max depth of 15, the accuracy is 0.95
with a max depth of 16, the accuracy is 0.95
with a max depth of 17, the accuracy is 0.95
with a max depth of 18, the accuracy is 0.95
with a max depth of 19, the accuracy is 0.95
with a max depth of 20, the accuracy is 0.95


In [45]:
# best model depth is 13

In [46]:
stats = []

for x in range(1,11):
    rf = RandomForestClassifier(min_samples_leaf=x, max_depth=11-x, random_state=123)

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_train)

    acc = rf.score(X_train, y_train)
    acc_val = rf.score(X_validate, y_validate)
    
    stats.append([x, 11-x,round(acc,2),round(acc_val,2) ]) 
    
#     print(f'min_leaf_samples = {x}, max_depth = {11-x}, accuracy train = {round(acc,2

In [47]:
stats_df = pd.DataFrame(stats, columns =['min_leaf','max_depth','train_acc','val_acc'])
stats_df

,min_leaf,max_depth,train_acc,val_acc
0,1,10,0.95,0.77
1,2,9,0.91,0.80
2,3,8,0.88,0.83
3,4,7,0.86,0.84
4,5,6,0.85,0.84
5,6,5,0.83,0.83
6,7,4,0.83,0.84
7,8,3,0.83,0.83
8,9,2,0.80,0.80
9,10,1,0.78,0.77


In [48]:
stats_df.sort_values('val_acc', ascending=False)

,min_leaf,max_depth,train_acc,val_acc
3,4,7,0.86,0.84
4,5,6,0.85,0.84
6,7,4,0.83,0.84
2,3,8,0.88,0.83
5,6,5,0.83,0.83
7,8,3,0.83,0.83
1,2,9,0.91,0.80
8,9,2,0.80,0.80
0,1,10,0.95,0.77
9,10,1,0.78,0.77


In [ ]:
#KNN

In [53]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [54]:
y_pred = knn.predict(X_train)
y_pred[:10]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

In [55]:
knn.score(X_train, y_train) #Default of n=5 performs best

0.8258426966292135

In [ ]:
#Evaluate your results using the model score, confusion matrix, and classification report.

In [56]:
pd.crosstab(y_train, y_pred)

col_0,0,1
survived,,
0,286,43
1,50,155


In [57]:
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_train, y_train)
y_pred = knn10.predict(X_train)
compute_class_metrics(y_train, y_pred)

Accuracy: 0.7883895131086143

True Positive Rate/Sensitivity/Recall/Power: 0.6536585365853659
False Positive Rate/False Alarm Ratio/Fall-out: 0.1276595744680851
True Negative Rate/Specificity/Selectivity: 0.8723404255319149
False Negative Rate/Miss Rate: 0.3463414634146341

Precision/PPV: 0.7613636363636364
F1 Score: 0.7034120734908137

Support (0): 205
Support (1): 329


In [58]:
knn20 = KNeighborsClassifier(n_neighbors=20)
knn20.fit(X_train, y_train)
y_pred = knn20.predict(X_train)
compute_class_metrics(y_train, y_pred)

Accuracy: 0.7340823970037453

True Positive Rate/Sensitivity/Recall/Power: 0.5317073170731708
False Positive Rate/False Alarm Ratio/Fall-out: 0.1398176291793313
True Negative Rate/Specificity/Selectivity: 0.8601823708206687
False Negative Rate/Miss Rate: 0.4682926829268293

Precision/PPV: 0.7032258064516129
F1 Score: 0.6055555555555556

Support (0): 205
Support (1): 329


In [59]:
#. model with 5 neighbors was more accurate

In [63]:
#Which model performs best on our out-of-sample data from validate?

In [60]:
knn.score(X_train, y_train)

0.8258426966292135

In [61]:
knn10.score(X_train, y_train)


0.7883895131086143

In [62]:
knn20.score(X_train, y_train)

0.7340823970037453

In [ ]:
#Logistic Regression

In [64]:
X_train.head()

,pclass,sibsp,parch,fare,alone,gender,Queenstown,Southampton
776,3,0,0,7.7500,1,0,1,0
829,1,0,0,80.0000,1,1,0,1
215,1,1,0,113.2750,0,1,0,0
258,1,0,0,512.3292,1,1,0,0
129,3,0,0,6.9750,1,0,0,1


In [66]:
features = ['fare','pclass']
X_train[features].head()

,fare,pclass
776,7.7500,3
829,80.0000,1
215,113.2750,1
258,512.3292,1
129,6.9750,3


In [69]:
#make it
lr = LogisticRegression()

#fit it
lr.fit(X_train[features], y_train)

LogisticRegression()

In [72]:
lr = LogisticRegression()
lr.fit(X_train[features], y_train)

LogisticRegression()

In [73]:
lr.score(X_train[features], y_train)

0.6722846441947565

In [77]:
features2 = ['fare','pclass', 'gender']

In [78]:
#make it
lr1 = LogisticRegression()

#fit it
lr1.fit(X_train[features2], y_train)

LogisticRegression()

In [80]:
lr1.score(X_train[features2], y_train)

0.8033707865168539

In [ ]:
#Gender made big difference in survival rate